In [1]:
from smt_model import solve_mcp_smt
from mip_utils import parse_instance

In [2]:
instance = parse_instance("/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/instances/inst07.dat")

In [3]:
solution = solve_mcp_smt(instance)

Starting Z3 Optimize check (Timeout: 300s, Symmetry: False)...
  UNKNOWN (Z3 Optimize timed out or encountered an issue)


In [4]:
solution

{'time': 1, 'optimal': False, 'obj': -1, 'sol': [[], [], [], [], [], []]}